使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第七章第一節 給湯設備）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from functools import lru_cache

# import section7_1_b as default
import s07_01_c_gas_boiler as gas
import s07_01_d_kerosene_boiler as oil
import s07_01_e_electric_heatpump as eheatpump
import s07_01_f_electric_heater as eheater
import s07_01_g_hybrid_hw_hybrid_ht_gas as hybrid_gas
# import section7_1_g_3 as hybrid_gas_3
import s07_01_h_hybrid_hw_gas_ht_hybrid as gas_hybrid
import s07_01_i_hybrid_hw_hybrid_ht_hybrid as whybrid
import s07_01_j_hot_water_saving_coefficient as watersaving
import s07_01_m_schedule_for_hot_water_use_and_bathing as schedule

import s09_02_water_based_solar_system as lss
import s09_03_air_based_solar_system as ass

from s11_01_external_condition import load_outdoor, get_Theta_ex
from s11_02_solar_radiation import load_solrad
from s11_03_life_schedule import load_schedule, get_schedule_hw

import copy

# 第七章　給湯設備

## 第一節　給湯設備

1．適用範囲～4．記号及び単位は省略

### 5. 給湯設備によるエネルギー消費量

#### 5.1 消費電力量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{E,W,d,t} = E_{E,hs,d,t} + E_{E,aux,ss,d,t}
$$

$ E_{E,W,d,t} $：日付$d$の時刻$t$における 1時間当たりの給湯設備の消費電力量 (kWh/h)  
$ E_{E,aux,ss,d,t} $：日付$d$の時刻$t$における 1時間当たりの太陽熱利用設備の補機の消費電力量 (kWh/h)    
$ E_{E,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機（給湯温水暖房機における給湯部も含む。以下、同じ。）の消費電力量 (kWh/h)  

日付$d$の時刻$t$における1時間当たりの太陽熱利用設備の補機の消費電力量$ E_{E,aux,ss,d,t} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

In [ ]:
# 1時間当たりの給湯設備の消費電力量 (1)
def get_E_E_W_d_t(n_p, L_HWH, heating_flag_d, region, sol_region, HW, SHC):
    """
    :param n_p: 仮想居住人数 (人)
    :param L_HWH: 温水暖房用熱源機の熱負荷
    :param heating_flag_d: 暖房日
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :param HW: 給湯機の仕様
    :param SHC: 集熱式太陽熱利用設備の仕様
    :return: 1日当たりの給湯設備の消費電力量 (kWh/d)
    """
    if HW is None or HW['hw_type'] is None:
        # 台所、洗面所及 び浴室等がいずれも無い場合は0とする
        return np.zeros(24 * 365)

    if HW['hw_type'] == 'コージェネレーションを使用する':
        return np.zeros(24 * 365)

    # ふろ機能の修正
    bath_function = get_normalized_bath_function(HW['hw_type'], HW['bath_function'])

    # 給湯負荷の生成
    args = {
        'n_p': n_p,
        'region': region,
        'sol_region': sol_region,
        'has_bath': HW['has_bath'],
        'bath_function': bath_function,
        'pipe_diameter': HW['pipe_diameter'],
        'kitchen_watersaving_A': HW['kitchen_watersaving_A'],
        'kitchen_watersaving_C': HW['kitchen_watersaving_C'],
        'shower_watersaving_A': HW['shower_watersaving_A'],
        'shower_watersaving_B': HW['shower_watersaving_B'],
        'washbowl_watersaving_C': HW['washbowl_watersaving_C'],
        'bath_insulation': HW['bath_insulation']
    }
    if SHC is not None:
        if SHC['type'] == '液体集熱式':
            args.update({
                'type': SHC['type'],
                'ls_type': SHC['ls_type'],
                'A_sp': SHC['A_sp'],
                'P_alpha_sp': SHC['P_alpha_sp'],
                'P_beta_sp': SHC['P_beta_sp'],
                'W_tnk_ss': SHC['W_tnk_ss']
            })
        elif SHC['type'] == '空気集熱式':
            args.update({
                'type': SHC['type'],
                'hotwater_use': SHC['hotwater_use'],
                'heating_flag_d': tuple(heating_flag_d),
                'A_col': SHC['A_col'],
                'P_alpha': SHC['P_alpha'],
                'P_beta': SHC['P_beta'],
                'V_fan_P0': SHC['V_fan_P0'],
                'm_fan_test': SHC['m_fan_test'],
                'd0': SHC['d0'],
                'd1': SHC['d1'],
                'W_tnk_ass': SHC['W_tnk_ass']
            })
        else:
            raise ValueError(SHC['type'])

    hotwater_load = get_hotwater_load(**args)

    # 1時間当たりの給湯機の消費電力量 (kWh/h)
    E_E_hs_d_t = get_E_E_hs_d_t(
        hw_type=HW['hw_type'],
        bath_function=bath_function,
        hybrid_category=HW['hybrid_category'],
        package_id=HW['package_id'] if 'package_id' in HW else None,
        e_rtd=HW['e_rtd'],
        e_dash_rtd=HW['e_dash_rtd'],
        m1see=HW['m1see'] if 'm1see' in HW else None,
        L_dashdash_k_d_t=hotwater_load['L_dashdash_k_d_t'],
        L_dashdash_s_d_t=hotwater_load['L_dashdash_s_d_t'],
        L_dashdash_w_d_t=hotwater_load['L_dashdash_w_d_t'],
        L_dashdash_b1_d_t=hotwater_load['L_dashdash_b1_d_t'],
        L_dashdash_b2_d_t=hotwater_load['L_dashdash_b2_d_t'],
        L_dashdash_ba1_d_t=hotwater_load['L_dashdash_ba1_d_t'],
        L_dashdash_ba2_d_t=hotwater_load['L_dashdash_ba2_d_t'],
        W_dash_k_d_t=hotwater_load['W_dash_k_d_t'],
        W_dash_s_d_t=hotwater_load['W_dash_s_d_t'],
        W_dash_w_d_t=hotwater_load['W_dash_w_d_t'],
        W_dash_b1_d_t=hotwater_load['W_dash_b1_d_t'],
        W_dash_b2_d_t=hotwater_load['W_dash_b2_d_t'],
        W_dash_ba1_d_t=hotwater_load['W_dash_ba1_d_t'],
        Theta_ex_Ave_d=hotwater_load['Theta_ex_Ave_d'],
        Theta_ex_Nave_d=hotwater_load['Theta_ex_Nave_d'],
        L_HWH=L_HWH
    )

    # 太陽利用設備の補機の消費電力量
    E_E_aux_ss_d_t = get_E_E_aux_ss_d_t(
        SHC=SHC,
        region=region,
        sol_region=sol_region,
        heating_flag_d=heating_flag_d
    )

    # 1時間当たりの給湯設備の消費電力量(1)
    E_E_W_d_t = E_E_hs_d_t + E_E_aux_ss_d_t

    return E_E_W_d_t

# 1時間当たりの補機の消費電力量 (kWh/h)
def get_E_E_aux_ss_d_t(SHC, region=None, sol_region=None, heating_flag_d=None):
    """
    :param SHC: 太陽熱利用設備の仕様
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分
    :param heating_flag_d: 暖房日
    :return: 1時間当たりの補機の消費電力量 (kWh/h)
    """
    if SHC is None:
        return np.zeros(24 * 365)
    elif SHC['type'] == '液体集熱式':
        # 第九章「自然エネルギー利用設備」第二節「液体集熱式太陽熱利用設備」の算定方法により定まる
        # 1時間当たりの補機の消費電力量 (kWh/h)
        return lss.get_E_E_lss_aux_d_t(
            ls_type=SHC['ls_type'],
            pmp_type='上記以外の機種',
            P_alpha_sp=SHC['P_alpha_sp'],
            P_beta_sp=SHC['P_beta_sp'],
            region=region,
            sol_region=sol_region
        )
    elif SHC['type'] == '空気集熱式':
        # 第九章「自然エネルギー利用設備」第三節「空気集熱式太陽熱利用設備」の算定方法により定まる
        # 1時間当たりの補機の消費電力量のうちの給湯設備への付加分 (kWh/h)
        return ass.calc_E_E_W_aux_ass_d_t(
            hotwater_use=SHC['hotwater_use'],
            heating_flag_d=heating_flag_d,
            region=region,
            sol_region=sol_region,
            P_alpha=SHC['P_alpha'],
            P_beta=SHC['P_beta'],
            A_col=SHC['A_col'],
            V_fan_P0=SHC['V_fan_P0'],
            m_fan_test=SHC['m_fan_test'],
            d0=SHC['d0'],
            d1=SHC['d1'],
            fan_sso=SHC['fan_sso'],
            fan_type=SHC['fan_type'],
            pump_sso=SHC['pump_sso']
        )
    else:
        raise ValueError(SHC['type'])

#### 5.2 ガス消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{G,W,d,t} = E_{G,hs,d,t}
$$

$ E_{G,W,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯設備のガス消費量 (MJ/h)  
$ E_{G,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機のガス消費量 (MJ/h)  

In [ ]:
# 1時間当たりの給湯設備のガス消費量 (MJ/h) (2)
def get_E_G_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, HW, SHC):
    """
    :param n_p:
    :param L_HWH:
    :param A_A:
    :param region:
    :param sol_region:
    :param HW:
    :param SHC: 集熱式太陽熱利用設備の仕様
    :return:
    """
    if HW is None or HW['hw_type'] is None:
        # 台所、洗面所及 び浴室等がいずれも無い場合は0とする
        return np.zeros(24 * 365)

    # ふろ機能の修正
    bath_function = get_normalized_bath_function(HW['hw_type'], HW['bath_function'])

    # 給湯負荷の生成
    args = {
        'n_p': n_p,
        'region': region,
        'sol_region': sol_region,
        'has_bath': HW['has_bath'],
        'bath_function': bath_function,
        'pipe_diameter': HW['pipe_diameter'],
        'kitchen_watersaving_A': HW['kitchen_watersaving_A'],
        'kitchen_watersaving_C': HW['kitchen_watersaving_C'],
        'shower_watersaving_A': HW['shower_watersaving_A'],
        'shower_watersaving_B': HW['shower_watersaving_B'],
        'washbowl_watersaving_C': HW['washbowl_watersaving_C'],
        'bath_insulation': HW['bath_insulation']
    }
    if SHC is not None:
        if SHC['type'] == '液体集熱式':
            args.update({
                'type': SHC['type'],
                'ls_type': SHC['ls_type'],
                'A_sp': SHC['A_sp'],
                'P_alpha_sp': SHC['P_alpha_sp'],
                'P_beta_sp': SHC['P_beta_sp'],
                'W_tnk_ss': SHC['W_tnk_ss']
            })
        elif SHC['type'] == '空気集熱式':
            args.update({
                'type': SHC['type'],
                'hotwater_use': SHC['hotwater_use'],
                'heating_flag_d': tuple(heating_flag_d),
                'A_col': SHC['A_col'],
                'P_alpha': SHC['P_alpha'],
                'P_beta': SHC['P_beta'],
                'V_fan_P0': SHC['V_fan_P0'],
                'm_fan_test': SHC['m_fan_test'],
                'd0': SHC['d0'],
                'd1': SHC['d1'],
                'W_tnk_ass': SHC['W_tnk_ass']
            })
        else:
            raise ValueError(SHC['type'])

    hotwater_load = get_hotwater_load(**args)

    # 1日当たりの給湯機のガス消費量
    E_G_hs_d = get_E_G_hs_d(
        hw_type=HW['hw_type'],
        hybrid_category=HW['hybrid_category'],
        e_rtd=HW['e_rtd'],
        e_dash_rtd=HW['e_dash_rtd'],
        bath_function=bath_function,
        package_id=HW['package_id'] if 'package_id' in HW else None,
        L_dashdash_k_d_t=hotwater_load['L_dashdash_k_d_t'],
        L_dashdash_s_d_t=hotwater_load['L_dashdash_s_d_t'],
        L_dashdash_w_d_t=hotwater_load['L_dashdash_w_d_t'],
        L_dashdash_b1_d_t=hotwater_load['L_dashdash_b1_d_t'],
        L_dashdash_b2_d_t=hotwater_load['L_dashdash_b2_d_t'],
        L_dashdash_ba1_d_t=hotwater_load['L_dashdash_ba1_d_t'],
        L_dashdash_ba2_d_t=hotwater_load['L_dashdash_ba2_d_t'],
        W_dash_k_d_t=hotwater_load['W_dash_k_d_t'],
        W_dash_s_d_t=hotwater_load['W_dash_s_d_t'],
        W_dash_w_d_t=hotwater_load['W_dash_w_d_t'],
        W_dash_b1_d_t=hotwater_load['W_dash_b1_d_t'],
        W_dash_b2_d_t=hotwater_load['W_dash_b2_d_t'],
        W_dash_ba1_d_t=hotwater_load['W_dash_ba1_d_t'],
        Theta_ex_Ave=hotwater_load['Theta_ex_Ave_d'],
        Theta_ex_Nave=hotwater_load['Theta_ex_Nave_d'],
        L_HWH=L_HWH
    )

    return E_G_hs_d

#### 5.3 灯油消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{K,W,d,t} = E_{K,hs,d,t}
$$

$ E_{K,W,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯設備の灯油消費量 (MJ/h)  
$ E_{K,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機の灯油消費量 (MJ/h)  

In [ ]:
# 1時間当たりの給湯設備の灯油消費量 (MJ/h) (3)
def get_E_K_W_d_t(n_p, L_HWH, heating_flag_d, A_A, region, sol_region, HW, SHC):
    """
    :param n_p:
    :param L_HWH:
    :param A_A:
    :param region:
    :param sol_region:
    :param HW:
    :param SHC: 集熱式太陽熱利用設備の仕様
    :return:
    """
    if HW is None or HW['hw_type'] is None:
        # 台所、洗面所及 び浴室等がいずれも無い場合は0とする
        return np.zeros(24 * 365)

    # ふろ機能の修正
    bath_function = get_normalized_bath_function(HW['hw_type'], HW['bath_function'])

    # 給湯負荷の生成
    args = {
        'n_p': n_p,
        'region': region,
        'sol_region': sol_region,
        'has_bath': HW['has_bath'],
        'bath_function': bath_function,
        'pipe_diameter': HW['pipe_diameter'],
        'kitchen_watersaving_A': HW['kitchen_watersaving_A'],
        'kitchen_watersaving_C': HW['kitchen_watersaving_C'],
        'shower_watersaving_A': HW['shower_watersaving_A'],
        'shower_watersaving_B': HW['shower_watersaving_B'],
        'washbowl_watersaving_C': HW['washbowl_watersaving_C'],
        'bath_insulation': HW['bath_insulation']
    }
    if SHC is not None:
        if SHC['type'] == '液体集熱式':
            args.update({
                'type': SHC['type'],
                'ls_type': SHC['ls_type'],
                'A_sp': SHC['A_sp'],
                'P_alpha_sp': SHC['P_alpha_sp'],
                'P_beta_sp': SHC['P_beta_sp'],
                'W_tnk_ss': SHC['W_tnk_ss']
            })
        elif SHC['type'] == '空気集熱式':
            args.update({
                'type': SHC['type'],
                'hotwater_use': SHC['hotwater_use'],
                'heating_flag_d': tuple(heating_flag_d),
                'A_col': SHC['A_col'],
                'P_alpha': SHC['P_alpha'],
                'P_beta': SHC['P_beta'],
                'V_fan_P0': SHC['V_fan_P0'],
                'm_fan_test': SHC['m_fan_test'],
                'd0': SHC['d0'],
                'd1': SHC['d1'],
                'W_tnk_ass': SHC['W_tnk_ass']
            })
        else:
            raise ValueError(SHC['type'])

    hotwater_load = get_hotwater_load(**args)

    # 1時間当たりの給湯機の灯油消費量 (MJ/h)
    E_k_hs_d_t = get_E_K_hs_d_t(
        hw_type=HW['hw_type'],
        e_rtd=HW['e_rtd'],
        e_dash_rtd=HW['e_dash_rtd'],
        bath_function=bath_function,
        L_dashdash_k_d_t=hotwater_load['L_dashdash_k_d_t'],
        L_dashdash_s_d_t=hotwater_load['L_dashdash_s_d_t'],
        L_dashdash_w_d_t=hotwater_load['L_dashdash_w_d_t'],
        L_dashdash_b1_d_t=hotwater_load['L_dashdash_b1_d_t'],
        L_dashdash_b2_d_t=hotwater_load['L_dashdash_b2_d_t'],
        L_dashdash_ba1_d_t=hotwater_load['L_dashdash_ba1_d_t'],
        L_dashdash_ba2_d_t=hotwater_load['L_dashdash_ba2_d_t'],
        Theta_ex_Ave_d=hotwater_load['Theta_ex_Ave_d']
    )

    return E_k_hs_d_t

#### 5.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における1時間当たりの給湯設備のその他の燃料による一次エネルギー消費量$E_{M,w,d,t}$は0とする。 

In [ ]:
# 1時間当たりの給湯設備のその他の燃料による一次エネルギー消費量
def get_E_M_W_d_t():
    # 1時間当たりの給湯設備のその他の燃料による一次エネルギー消費量は0とする
    return np.zeros(24 * 365)

### 6. 給湯機のエネルギー消費量

省略

In [ ]:
# 1時間当たりの給湯機の消費電力量 (kWh/h)
def get_E_E_hs_d_t(hw_type, bath_function, package_id, hybrid_category, e_rtd, e_dash_rtd, m1see, Theta_ex_Nave_d, W_dash_k_d_t, W_dash_s_d_t,
                   W_dash_w_d_t,
                   W_dash_b1_d_t,
                   W_dash_b2_d_t, W_dash_ba1_d_t, Theta_ex_Ave_d, L_dashdash_k_d_t, L_dashdash_s_d_t, L_dashdash_w_d_t,
                   L_dashdash_b1_d_t,
                   L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t, L_HWH):
    """
    :param hw_type: 給湯機／給湯温水暖房機の種類
    :param bath_function: 給湯機の種類
    :param hybrid_category: 電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機の区分
    :param package_id: パッケージID
    :param e_rtd: 当該給湯機の効率
    :param e_dash_rtd: ：「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）に定義される「エネルギー消費効率」
    :param m1see: M1スタンダードに基づくJIS相当効率
    :param Theta_ex_Nave_d: 夜間平均外気温 (℃)
    :param W_dash_k_d_t: 1日当たりの台所水栓における節湯補正給湯量 (L/d)
    :param W_dash_s_d_t: 1日当たりの浴室シャワー水栓における節湯補正給湯量 (L/d)
    :param W_dash_w_d_t: 1日当たりの洗面水栓における節湯補正給湯量 (L/d)
    :param W_dash_b1_d_t: 1日当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/d)
    :param W_dash_b2_d_t: 1日当たりの浴槽追焚時における節湯補正給湯量 (L/d)
    :param W_dash_ba1_d_t: 1日当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/d)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_k_d_t: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_s_d_t: 1日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_w_d_t: 1日当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_b1_d_t: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_b2_d_t: 1日当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_ba1_d_t: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_ba2_d_t: 1日当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_HWH: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :return: 1時間当たりの給湯機の消費電力量 (MJ/h)
    """
    if hw_type == 'ガス従来型給湯機' or hw_type == 'ガス従来型給湯温水暖房機' \
            or hw_type == 'ガス潜熱回収型給湯機' or hw_type == 'ガス潜熱回収型給湯温水暖房機':
        return gas.get_E_E_hs_d_t(
            W_dash_k_d_t=W_dash_k_d_t,
            W_dash_s_d_t=W_dash_s_d_t,
            W_dash_w_d_t=W_dash_w_d_t,
            W_dash_b1_d_t=W_dash_b1_d_t,
            W_dash_b2_d_t=W_dash_b2_d_t,
            W_dash_ba1_d_t=W_dash_ba1_d_t,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '石油従来型給湯機' or hw_type == '石油従来型給湯温水暖房機' \
            or hw_type == '石油潜熱回収型給湯機' or hw_type == '石油潜熱回収型給湯温水暖房機':
        return oil.get_E_E_hs_d_t(W_dash_k_d_t=W_dash_k_d_t, W_dash_s_d_t=W_dash_s_d_t, W_dash_w_d_t=W_dash_w_d_t,
                                  W_dash_b1_d_t=W_dash_b1_d_t, W_dash_ba1_d_t=W_dash_ba1_d_t,
                                  W_dash_b2_d_t=W_dash_b2_d_t, Theta_ex_Ave_d=Theta_ex_Ave_d,
                                  L_dashdash_ba2_d_t=L_dashdash_ba2_d_t)
    elif hw_type == '電気ヒートポンプ給湯機':
        return eheatpump.get_E_E_hs_d_t(
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t,
            e_rtd=e_rtd,
            m1see=m1see,
            Theta_ex_Nave=Theta_ex_Nave_d
        )
    elif hw_type == '電気ヒーター給湯機' or hw_type == '電気ヒーター給湯温水暖房機':
        return eheater.get_E_E_hs_d_t(
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t,
            Theta_ex_Ave_d=Theta_ex_Ave_d
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(仕様による)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(仕様による)':
        return hybrid_gas.get_E_E_hs_d_t(
            hybrid_category=hybrid_category,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(試験された値を用いる)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(試験された値を用いる)':
        return hybrid_gas.get_E_E_hs_d_t(
            bath_function=bath_function,
            package_id=package_id,
            W_dash_ba1_d_t=W_dash_ba1_d_t,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return gas_hybrid.get_E_E_hs(
            W_dash_k_d_t=W_dash_k_d_t,
            W_dash_s_d_t=W_dash_s_d_t,
            W_dash_w_d_t=W_dash_w_d_t,
            W_dash_b1_d_t=W_dash_b1_d_t,
            W_dash_b2_d_t=W_dash_b2_d_t,
            W_dash_ba1_d_t=W_dash_ba1_d_t,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return whybrid.get_E_E_hs_d_t(
            L_HWH=L_HWH,
            hybrid_category=hybrid_category,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    else:
        raise ValueError(hw_type)


# 1日当たりの給湯機のガス消費量
def get_E_G_hs_d(hw_type, hybrid_category, e_rtd, e_dash_rtd, bath_function, package_id, Theta_ex_Nave, W_dash_k_d_t, W_dash_s_d_t,
                 W_dash_w_d_t, W_dash_b1_d_t, W_dash_b2_d_t, W_dash_ba1_d_t, Theta_ex_Ave, L_dashdash_k_d_t,
                 L_dashdash_s_d_t, L_dashdash_w_d_t,
                 L_dashdash_b1_d_t, L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t, L_HWH):
    """
    :param hw_type: 給湯機／給湯温水暖房機の種類
    :param hybrid_category: 電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機の区分
    :param e_rtd: 当該給湯機の効率
    :param e_dash_rtd: ：「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）に定義される「エネルギー消費効率」
    :param bath_function: ふろ機能の種類
    :param Theta_ex_Nave: 夜間平均外気温 (℃)
    :param W_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯量 (L/h)
    :param W_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯量 (L/h)
    :param W_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯量 (L/h)
    :param W_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h)
    :param W_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯量 (L/h)
    :param W_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/h)
    :param Theta_ex_Ave: 日平均外気温度 (℃)
    :param L_dashdash_k_d: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_w_d: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d: 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba2_d: 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_HWH: 1日当たりの温水暖房の熱負荷 (MJ/d)
    :return: 1時間当たりの給湯機のガス消費量 (MJ/h)
    """
    if hw_type == 'ガス従来型給湯機' or hw_type == 'ガス従来型給湯温水暖房機' \
            or hw_type == 'ガス潜熱回収型給湯機' or hw_type == 'ガス潜熱回収型給湯温水暖房機':
        return gas.get_E_G_hs_d_t(
            hw_type=hw_type,
            e_rtd=e_rtd,
            e_dash_rtd=e_dash_rtd,
            Theta_ex_Ave_d=Theta_ex_Ave,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t,
            bath_function=bath_function
        )
    elif hw_type == '石油従来型給湯機' or hw_type == '石油従来型給湯温水暖房機' \
            or hw_type == '石油潜熱回収型給湯機' or hw_type == '石油潜熱回収型給湯温水暖房機':
        return oil.get_E_G_hs_d_t()
    elif hw_type == '電気ヒートポンプ給湯機':
        return eheatpump.get_E_G_hs_d_t()
    elif hw_type == '電気ヒーター給湯機' or hw_type == '電気ヒーター給湯温水暖房機':
        return eheater.get_E_G_hs()
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(仕様による)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(仕様による)':
        return hybrid_gas.get_E_G_hs_d_t(
            hybrid_category=hybrid_category,
            Theta_ex_Ave_d=Theta_ex_Ave,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(試験された値を用いる)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(試験された値を用いる)':
        return hybrid_gas.get_E_G_hs_d_t(
            bath_function=bath_function,
            package_id=package_id,
            Theta_ex_Ave_d=Theta_ex_Ave,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t,
            W_dash_ba1_d_t=W_dash_ba1_d_t
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return gas_hybrid.get_E_G_hs(
            Theta_ex_Ave=Theta_ex_Ave,
            L_dashdash_k=L_dashdash_k_d_t,
            L_dashdash_s=L_dashdash_s_d_t,
            L_dashdash_w=L_dashdash_w_d_t,
            L_dashdash_b1=L_dashdash_b1_d_t,
            L_dashdash_b2=L_dashdash_b2_d_t,
            L_dashdash_ba1=L_dashdash_ba1_d_t,
            L_dashdash_ba2=L_dashdash_ba2_d_t,
            bath_function=bath_function
        )
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return whybrid.get_E_G_hs_d_t(
            L_HWH=L_HWH,
            hybrid_category=hybrid_category,
            Theta_ex_Ave=Theta_ex_Ave,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == 'コージェネレーションを使用する':
        return np.zeros(365)
    else:
        raise ValueError(hw_type)


# 1時間当たりの給湯機の灯油消費量 (MJ/h)
def get_E_K_hs_d_t(hw_type, e_rtd, e_dash_rtd, bath_function, Theta_ex_Ave_d, L_dashdash_k_d_t, L_dashdash_s_d_t,
                   L_dashdash_w_d_t,
                   L_dashdash_b1_d_t, L_dashdash_b2_d_t, L_dashdash_ba1_d_t, L_dashdash_ba2_d_t):
    """
    :param hw_type: 給湯機／給湯温水暖房機の種類
    :param e_rtd: 当該給湯機の効率
    :param e_dash_rtd: ：「エネルギーの使用の合理化に関する法律」に基づく「特定機器の性能の向上に関する製造事業者等の 判断の基準等」（ガス温水機器）に定義される「エネルギー消費効率」
    :param bath_function: ふろ機能の種類
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの給湯機の灯油消費量 (MJ/h)
    """
    if hw_type == 'ガス従来型給湯機' or hw_type == 'ガス従来型給湯温水暖房機' \
            or hw_type == 'ガス潜熱回収型給湯機' or hw_type == 'ガス潜熱回収型給湯温水暖房機':
        return gas.get_E_K_hs_d_t()
    elif hw_type == '石油従来型給湯機' or hw_type == '石油従来型給湯温水暖房機' \
            or hw_type == '石油潜熱回収型給湯機' or hw_type == '石油潜熱回収型給湯温水暖房機':
        return oil.get_E_K_hs_d_t(
            hw_type=hw_type,
            bath_function=bath_function,
            e_rtd=e_rtd,
            e_dash_rtd=e_dash_rtd,
            Theta_ex_Ave_d=Theta_ex_Ave_d,
            L_dashdash_k_d_t=L_dashdash_k_d_t,
            L_dashdash_s_d_t=L_dashdash_s_d_t,
            L_dashdash_w_d_t=L_dashdash_w_d_t,
            L_dashdash_b1_d_t=L_dashdash_b1_d_t,
            L_dashdash_b2_d_t=L_dashdash_b2_d_t,
            L_dashdash_ba1_d_t=L_dashdash_ba1_d_t,
            L_dashdash_ba2_d_t=L_dashdash_ba2_d_t
        )
    elif hw_type == '電気ヒートポンプ給湯機':
        return eheatpump.get_E_K_hs_d_t()
    elif hw_type == '電気ヒーター給湯機' or hw_type == '電気ヒーター給湯温水暖房機':
        return eheater.get_E_K_hs()
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(仕様による)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(仕様による)':
        return gas_hybrid.get_E_K_hs()
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(試験された値を用いる)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(試験された値を用いる)':
        return hybrid_gas.get_E_K_hs_d_t()
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return hybrid_gas.get_E_K_hs_d_t()
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return whybrid.get_E_K_hs_d_t()
    elif hw_type == 'コージェネレーションを使用する':
        return np.zeros(365)
    else:
        raise ValueError(hw_type)


# 表4 評価可能な給湯機／給湯温水暖房機の種類
def get_normalized_bath_function(hw_type, bath_function):
    if hw_type == 'ガス従来型給湯機' or hw_type == 'ガス従来型給湯温水暖房機' \
            or hw_type == 'ガス潜熱回収型給湯機' or hw_type == 'ガス潜熱回収型給湯温水暖房機':
        return bath_function
    elif hw_type == '石油従来型給湯機' or hw_type == '石油従来型給湯温水暖房機' \
            or hw_type == '石油潜熱回収型給湯機' or hw_type == '石油潜熱回収型給湯温水暖房機':
        return bath_function
    elif hw_type == '電気ヒートポンプ給湯機':
        return bath_function
    elif hw_type == '電気ヒーター給湯機' or hw_type == '電気ヒーター給湯温水暖房機':
        return bath_function
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(試験された値を用いる)' \
            or hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(仕様による)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(試験された値を用いる)' \
            or hw_type == '電気ヒートポンプ・ガス併用型給湯機(仕様による)':
        return "ふろ給湯機(追焚あり)"
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return "ふろ給湯機(追焚あり)"
    elif hw_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return "ふろ給湯機(追焚あり)"
    elif hw_type == 'コージェネレーションを使用する':
        return bath_function
    else:
        raise ValueError(hw_type)

### 7. 太陽熱補正給湯熱負荷

$$\large
\displaystyle
L''_{k,d,t} = L'_{k,d,t} - L_{sun,d,t} \times \frac{L'_{k,d,t}}{(L'_{k,d,t}+L'_{s,d,t}+L'_{w,d,t}+L'_{b1,d,t}+L'_{b2,d,t}+L'_{ba1,d,t})}
$$


$$\large
\displaystyle L''_{s,d,t} = L'_{s,d,t} - L_{sun,d,t} \times \frac { L'_{s,d,t}}{ (L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{w,d,t} = L'_{w,d,t} - L_{sun,d,t} \times \frac { L'_{w,d,t}}{ (L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$  


$$\large
\displaystyle L''_{b1,d,t}= L'_{b1,d,t} - L_{sun,d,t} \times \frac{ L'_{b1,d,t}} {(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{b2,d,t}= L'_{b2,d,t} - L_{sun,d,t} \times \frac{ L'_{b2,d,t}} {(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{ba1,d,t} = L'_{ba1,d,t} - L_{sun,d,t} \times \frac{L'_{ba1,d,t}}{(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t})}
$$

$$\large
\displaystyle L''_{ba2,d,t} = L'_{ba2,d,t}
$$

$ L''_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h)  
$ L'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/h)  
$ L_{sun,d,t} $：日付$d$の時刻$t$における1時間当たりの太陽熱利用給湯設備による補正集熱量(MJ/h)

日付$d$の時刻$t$における1時間当たりの太陽熱利用給湯設備による補正集熱量$ L_{sun,d,t} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

In [ ]:
# 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h) (4a)
def get_L_dashdash_k_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                         L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    """

    L_dashdash_k_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_k_d_t[f] = L_dash_k_d_t[f] - L_sun_d_t[f] * (L_dash_k_d_t[f] / L_dash_d_t[f])

    return L_dashdash_k_d_t


# 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h) (4b)
def get_L_dashdash_s_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                         L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 (MJ/h)
    """
    L_dashdash_s_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_s_d_t[f] = L_dash_s_d_t[f] - L_sun_d_t[f] * (L_dash_s_d_t[f] / L_dash_d_t[f])

    return L_dashdash_s_d_t


# 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h) (4c)
def get_L_dashdash_w_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                         L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷 (MJ/h)
    """
    L_dashdash_w_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_w_d_t[f] = L_dash_w_d_t[f] - L_sun_d_t[f] * (L_dash_w_d_t[f] / L_dash_d_t[f])

    return L_dashdash_w_d_t


# 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h) (4d)
def get_L_dashdash_b1_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                          L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷 (MJ/h)
    """
    L_dashdash_b1_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_b1_d_t[f] = L_dash_b1_d_t[f] - L_sun_d_t[f] * (L_dash_b1_d_t[f] / L_dash_d_t[f])

    return L_dashdash_b1_d_t


# 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h) (4e)
def get_L_dashdash_b2_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                          L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    """
    L_dashdash_b2_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_b2_d_t[f] = L_dash_b2_d_t[f] - L_sun_d_t[f] * (L_dash_b2_d_t[f] / L_dash_d_t[f])

    return L_dashdash_b2_d_t


# 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h) (4f)
def get_L_dashdash_ba1_d_t(L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                           L_sun_d_t):
    """
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/hd)
    :param L_dash_b2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :param L_sun_d_t: 1時間当たりの太陽熱利用給湯設備による補正集熱量 (MJ/h)
    :return: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    """
    L_dashdash_ba1_d_t = np.zeros(24 * 365)

    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    f = L_dash_d_t > 0
    L_dashdash_ba1_d_t[f] = L_dash_ba1_d_t[f] - L_sun_d_t[f] * (L_dash_ba1_d_t[f] / L_dash_d_t[f])

    return L_dashdash_ba1_d_t


# 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h) (4g)
def get_L_dashdash_ba2_d_t(L_dash_ba2_d_t):
    """
    :param L_dash_ba2_d_t: 1時間当たりの浴槽追焚時における節湯補正給湯負荷 (MJ/h)
    :return: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    """
    return L_dash_ba2_d_t

In [ ]:
# 太陽熱利用給湯設備による補正集熱量
def get_L_sun_d_t(region, sol_region=None, solar_device=None, ls_type=None, A_sp=None, P_alpha_sp=None, P_beta_sp=None,
                  W_tnk_ss=None, hotwater_use=None, heating_flag_d=None, A_col=None, P_alpha=None, P_beta=None,
                  V_fan_P0=None, d0=None,
                  d1=None, m_fan_test=None, W_tnk_ass=None, Theta_wtr_d=None, L_dash_k_d_t=None, L_dash_s_d_t=None,
                  L_dash_w_d_t=None, L_dash_b1_d_t=None, L_dash_b2_d_t=None, L_dash_ba1_d_t=None):
    """
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分
    :param solar_device: 太陽熱利用設備の種類 (液体集熱式,空気集熱式,None)
    :param ls_type: 液体集熱式太陽熱利用設備の種類 (太陽熱温水器,ソーラーシステム)
    :param A_sp: 太陽熱集熱部の有効集熱面積 (m2)
    :param P_alpha_sp: 太陽熱集熱部の方位角 (°)
    :param P_beta_sp: 太陽熱集熱部の傾斜角 (°)
    :param W_tnk_ss: タンク容量 (L)
    :param W_tnk_ass: タンク容量 (L)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :param L_dash_k_d_t: 台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの太陽熱利用設備による補正集熱量 (MJ/h)
    """
    if solar_device == '液体集熱式':
        return lss.get_L_sun_lss_d_t(
            region=region,
            sol_region=sol_region,
            ls_type=ls_type,
            A_sp=A_sp,
            P_alpha_sp=P_alpha_sp,
            P_beta_sp=P_beta_sp,
            W_tnk_ss=W_tnk_ss,
            Theta_wtr_d=Theta_wtr_d,
            L_dash_k_d_t=L_dash_k_d_t,
            L_dash_s_d_t=L_dash_s_d_t,
            L_dash_w_d_t=L_dash_w_d_t,
            L_dash_b1_d_t=L_dash_b1_d_t,
            L_dash_b2_d_t=L_dash_b2_d_t,
            L_dash_ba1_d_t=L_dash_ba1_d_t
        )
    elif solar_device == '空気集熱式':
        if hotwater_use == True:
            outdoor = load_outdoor()
            Theta_ex_d_t = get_Theta_ex(region, outdoor)
            Theta_col_nonopg_d_t, Theta_col_opg_d_t = ass.calc_Theta_col(A_col, P_alpha, P_beta, V_fan_P0, d0, d1,
                                                                         m_fan_test, region, sol_region, Theta_ex_d_t)
            t_fan_d_t = ass.get_t_fan_d_t(Theta_col_nonopg_d_t, Theta_col_opg_d_t)
            t_cp_d_t = ass.get_t_cp_d_t(hotwater_use, t_fan_d_t, heating_flag_d)
            V_fan_d_t = ass.get_V_fan_d_t(t_fan_d_t, V_fan_P0)
            Q_col_d_t = ass.get_Q_col_d_t(V_fan_d_t, Theta_col_opg_d_t, Theta_ex_d_t)
            Q_d = ass.get_Q_d(Q_col_d_t, t_cp_d_t)
            L_tnk_d = ass.get_L_tnk_d(Q_d, W_tnk_ass, Theta_wtr_d)
            return ass.get_L_sun_ass_d_t(L_tnk_d, L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t,
                                         L_dash_b2_d_t, L_dash_ba1_d_t)
        else:
            return np.zeros(24 * 365)
    elif solar_device is None:
        return np.zeros(24 * 365)
    else:
        raise ValueError(solar_device)

### 8. 節湯補正給湯熱負荷

$$\large
\displaystyle L'_{k,d,t} = W'_{k,d,t} \times (\theta_{sw,k} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{s,d,t} = W'_{s,d,t} \times (\theta_{sw,s} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{w,d,t} = W'_{w,d,t} \times (\theta_{sw,w} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

In [ ]:
# 台所水栓における節湯補正給湯負荷 (MJ/h) (5a)
def get_L_dash_k_d_t(W_dash_k_d_t, Theta_sw_k, Theta_wtr_d):
    """
    :param W_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯量 (L/h)
    :param Theta_sw_k: 台所水栓における基給湯量 (℃)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 台所水栓における節湯補正給湯負荷 (MJ/h)
    """
    return W_dash_k_d_t * (Theta_sw_k - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)


# 浴室シャワー水栓における節湯補正給湯負荷 (5b)
def get_L_dash_s_d_t(W_dash_s_d_t, Theta_sw_s, Theta_wtr_d):
    """
    :param W_dash_s_d_t: 1時間当たりの浴室シャワーにおける節湯補正給湯量 (L/h)
    :param Theta_sw_k: 浴室シャワーにおける基給湯量 (℃)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 浴室シャワーにおける節湯補正給湯負荷 (MJ/h)
    """
    return W_dash_s_d_t * (Theta_sw_s - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)


# 洗面水栓における節湯補正給湯負荷 (5c)
def get_L_dash_w_d_t(W_dash_w_d_t, Theta_sw_w, Theta_wtr_d):
    """
    :param W_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯量 (L/d)
    :param Theta_sw_k: 洗面水栓における基給湯量 (℃)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 洗面水栓における節湯補正給湯負荷 (MJ/d)
    """
    return W_dash_w_d_t * (Theta_sw_w - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)

浴室等が無い場合（ふろ機能の種類によらない）

$$\large
\displaystyle L'_{b1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{b2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能の場合 

$$\large
\displaystyle L'_{b1,d,t} = W'_{b1,d,t} \times ( \theta_{sw,b1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{b2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚の有無によらない）の場合

$$\large
\displaystyle L'_{b1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{b2,d,t} = W'_{b2,d,t} \times ( \theta_{sw,b2} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

In [ ]:
# 浴槽水栓湯はり時における節水補正給湯熱負荷 L_dash_b1_d, L_dash_b2_d
def get_L_dash_bx_d_t(W_dash_b1_d_t, W_dash_b2_d_t, Theta_wtr_d, has_bath, bash_function):
    """
    :param W_dash_b1_d_t: 1日当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/d)
    :param W_dash_b2_d_t: 1日当たりの浴槽自動湯はり時における節湯補正給湯量 (L/d)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :param has_bath: 浴室用の有無
    :param bash_function: ふろ機能の種類
    :return: 浴槽水栓湯はり時・浴槽自動湯はり時における節水補正給湯熱負荷 (MJ/d)
    """
    if has_bath == False:
        L_dash_b1_d_t = np.zeros(24 * 365)  # (5-1d)
        L_dash_b2_d_t = np.zeros(24 * 365)  # (5-1e)
        return L_dash_b1_d_t, L_dash_b2_d_t
    elif bash_function == '給湯単機能':
        Theta_sw_b1 = get_Theta_sw_b1()
        L_dash_b1_d_t = W_dash_b1_d_t * (Theta_sw_b1 - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)  # (5-2d)
        L_dash_b2_d_t = np.zeros(24 * 365)  # (5-2e)
        return L_dash_b1_d_t, L_dash_b2_d_t
    elif bash_function == 'ふろ給湯機(追焚あり)' or bash_function == 'ふろ給湯機(追焚なし)':
        Theta_sw_b2 = get_Theta_sw_b2()
        L_dash_b1_d_t = np.zeros(24 * 365)  # (5-3d)
        L_dash_b2_d_t = W_dash_b2_d_t * (Theta_sw_b2 - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)  # (5-3e)
        return L_dash_b1_d_t, L_dash_b2_d_t
    else:
        raise ValueError(bash_function)

浴室等が無い場合

$$\large
\displaystyle L'_{ba1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能またはふろ給湯機（追焚なし）の場合 

$$\large
\displaystyle L'_{ba1,d,t} = W'_{ba1,d,t} \times ( \theta_{sw,ba1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{ba2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚あり）の場合

$$\large
\displaystyle L'_{ba1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d,t} = L_{ba,d,t} \times 1.25
$$

$ L'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/h)  
$ W'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯量(L/h)  
$ W'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯量(L/h)  
$ W'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯量(L/h)  
$ W'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯量(L/h)  
$ W'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯量(L/h)  
$ W'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯量(L/h)  
$ W'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯量(L/h)  
$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ \theta_{sw,k} $：台所水栓における基準給湯温度(℃)  
$ \theta_{sw,s} $：浴室シャワー水栓における基準給湯温度(℃)  
$ \theta_{sw,w} $：洗面水栓における基準給湯温度(℃)  
$ \theta_{sw,b1} $：浴槽水栓湯はり時における基準給湯温度(℃)  
$ \theta_{sw,b2} $：浴槽自動湯はり時における基準給湯温度(℃)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{sw,ba2} $：浴槽追焚時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)

In [ ]:
# 浴槽水栓さし湯時における節水補正給湯熱負荷 L_dash_ba1_d, L_dash_ba2_d
def get_L_dash_bax_d_t(W_dash_ba1_d_t, Theta_wtr_d, L_ba_d_t, has_bath, bash_function):
    """
    :param W_dash_ba1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :param L_ba_d_t: 1時間当たりの浴槽沸かし直しによる給湯熱負荷 (MJ/h)
    :param has_bath: 浴室等の有無
    :param bash_function: ふろ機能の種類 (給湯単機能,ふろ給湯機(追焚なし),ふろ給湯機(追焚あり))
    :return: 浴槽水栓さし湯時／浴槽追焚時における節水補正給湯熱負荷 (MJ/h)
    """
    if has_bath == False:
        L_dash_ba1_d_t = np.zeros(24 * 365)  # (5-1f)
        L_dash_ba2_d_t = np.zeros(24 * 365)  # (5-1g)
        return L_dash_ba1_d_t, L_dash_ba2_d_t
    elif bash_function == '給湯単機能' or bash_function == 'ふろ給湯機(追焚なし)':
        Theta_sw_ba1 = get_Theta_sw_ba1()
        L_dash_ba1_d_t = W_dash_ba1_d_t * (Theta_sw_ba1 - np.repeat(Theta_wtr_d, 24)) * 4.186 * 10 ** (-3)  # (5-2f)
        L_dash_ba2_d_t = np.zeros(24 * 365)  # (5-2g)
        return L_dash_ba1_d_t, L_dash_ba2_d_t
    elif bash_function == 'ふろ給湯機(追焚あり)':
        L_dash_ba1_d_t = np.zeros(24 * 365)  # (5-3f)
        L_dash_ba2_d_t = L_ba_d_t * 1.25  # (5-3g)
        return L_dash_ba1_d_t, L_dash_ba2_d_t
    else:
        raise ValueError(bash_function)

<div style="text-align: center;font-weight: bold;">表　用途ごとの基準給湯温度</div>

| 用途 |　記号　| 基準給湯温度(℃) |
| ----- | ---- | ---- |
| 台所水栓 | $\theta_{sw,k}$ | 40 |
|浴室シャワー水栓|$\theta_{sw,s}$|40|
|洗面水栓|$\theta_{sw_w}$|40|
|浴槽水栓湯はり|$\theta_{sw,b1}$|40|
|浴槽自動湯はり|$\theta_{sw,b1}$|40|
|浴槽水栓さし湯|$\theta_{sw,ba1}$|60|


In [ ]:
# 台所水栓の基準給湯温度
def get_Theta_sw_k():
    return table_5[0]


# 浴室シャワー水栓の基準給湯温度
def get_Theta_sw_s():
    return table_5[1]


# 洗面水栓の基準給湯温度
def get_Theta_sw_w():
    return table_5[2]


# 浴槽水栓湯はりの基準給湯温度
def get_Theta_sw_b1():
    return table_5[3]


# 浴槽自動湯はりの基準給湯温度
def get_Theta_sw_b2():
    return table_5[4]


# 浴槽水栓さし湯の基準給湯温度
def get_Theta_sw_ba1():
    return table_5[5]


# 表 5 用途ごとの基準給湯温度
table_5 = [
    40,
    40,
    40,
    40,
    40,
    60
]


### 9. 節湯補正給湯量

$$\large
\displaystyle W'_{k,d,t} = W_{k,d,t} \times f_{sk} \times f_{sp}
$$

$$\large
\displaystyle W'_{s,d,t} = W_{s,d,t} \times f_{ss} \times f_{sp}
$$

$$\large
\displaystyle W'_{w,d,t} = W_{w,d,t} \times f_{sw} \times f_{sp}
$$

$$\large
\displaystyle W'_{b1,d,t} = W_{b1,d,t} \times f_{sp} \times f_{sb}
$$

$$\large
\displaystyle W'_{b2,d,t} = W_{b2,d,t} \times f_{sb}
$$

$$\large
\displaystyle W'_{ba1,d,t} = W_{ba1,d,t} \times f_{sp}
$$

$ W'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯量(L/h)  
$ W'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯量(L/h)  
$ W'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯量(L/h)  
$ W'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯量(L/h)  
$ W'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯量(L/h)  
$ W'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯量(L/h)  
$ W'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯量(L/h)  
$ W_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における基準給湯量(L/h)  
$ W_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における基準給湯量(L/h)  
$ W_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における基準給湯量(L/h)  
$ W_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における基準給湯量(L/h)  
$ W_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における基準給湯量(L/h)  
$ W_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量(L/h)  
$ W_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における基準給湯量(L/h)  
$ f_{sk} $：台所水栓における節湯の効果係数(-)  
$ f_{ss} $：浴室シャワー水栓における節湯の効果係数(-)  
$ f_{sw} $：洗面水栓における節湯の効果係数(-)  
$ f_{sp} $：配管における節湯の効果係数(-)  
$ f_{sb} $：浴槽における節湯の効果係数(-)  

In [ ]:
# 1時間当たりの台所水栓における節湯補正給湯量 [L/h] (6a)
def get_W_dash_k_d_t(W_k_d_t, kitchen_watersaving_A, kitchen_watersaving_C, pipe_diameter):
    """
    :param W_k_d_t: 1時間当たりの台所水栓における基準給湯量 (L/h)
    :param kitchen_watersaving_A: 台所水栓の手元止水機能の有無
    :param kitchen_watersaving_C: 台所水栓の水優先吐水機能の有無
    :param pipe_diameter: ヘッダー分岐後の径
    :return: 1時間当たりの台所水栓における節湯補正給湯量 (L/h)
    """
    # 台所水栓における節湯の効果係数
    f_sk = watersaving.get_f_sk(kitchen_watersaving_A, kitchen_watersaving_C)

    # 配管における節湯の効果係数
    f_sp = watersaving.get_f_sp(pipe_diameter)

    return W_k_d_t * f_sk * f_sp


# 1時間当たりの浴室シャワーにおける節湯補正給湯量 (L/h) (6a)
def get_W_dash_s_d_t(W_s_d_t, shower_watersaving_A, shower_watersaving_B, pipe_diameter):
    """
    :param W_s_d_t: 浴室シャワーにおける基準給湯量 (L/h)
    :param shower_watersaving_A: 浴室シャワー水栓の手元止水機能の有無
    :param shower_watersaving_B: 浴室シャワー水栓の小流量吐水機能の有無
    :param pipe_diameter: ヘッダー分岐後の径
    :return: 1時間当たりの浴室シャワーにおける節湯補正給湯量 (L/h)
    """
    # 浴室シャワー水栓のける節湯の効果係数
    f_ss = watersaving.get_f_ss(shower_watersaving_A, shower_watersaving_B)

    # 配管における節湯の効果係数
    f_sp = watersaving.get_f_sp(pipe_diameter)

    return W_s_d_t * f_ss * f_sp


# 1時間当たりの台所水栓における節湯補正給湯量 (L/h) (6c)
def get_W_dash_w_d_t(W_w_d_t, washbowl_watersaving_C, pipe_diameter):
    """
    :param W_w_d_t: 台所水栓における基準給湯量 (L/h)
    :param washbowl_watersaving_C: 洗面水栓の水優先吐水機能の有無
    :param pipe_diameter: ヘッダー分岐後の径
    :return: 1時間当たりの台所水栓における節湯補正給湯量 (L/h)
    """
    # 配管における節湯の効果係数
    f_sp = watersaving.get_f_sp(pipe_diameter)

    # 洗面水栓における節湯の効果係数
    f_sw = watersaving.get_f_sw(washbowl_watersaving_C)

    return W_w_d_t * f_sw * f_sp


# 1時間当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h) (6d)
def get_W_dash_b1_d_t(W_b1_d_t, pipe_diameter):
    """
    :param W_b1_d_t: 浴槽水栓湯はり時における基準給湯量 (L/h)
    :param pipe_diameter: ヘッダー分岐後の径
    :return: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯量 (L/h)
    """
    # 配管における節湯の効果係数
    f_sp = watersaving.get_f_sp(pipe_diameter)

    # 浴槽における節湯の効果係数
    f_sb = watersaving.get_f_sb()

    return W_b1_d_t * f_sp * f_sb


# 1時間当たりの浴槽自動湯はり時における節湯補正給湯量 (L/h) (6e)
def get_W_dash_b2_d_t(W_b2_d_t):
    """
    :param W_b2_d_t: 浴槽自動湯はり時における基準給湯量 (L/h)
    :return: 1時間当たりの浴槽自動湯はり時における節湯補正給湯量 (L/h)
    """
    # 浴槽における節湯の効果係数
    f_sb = watersaving.get_f_sb()

    return W_b2_d_t * f_sb


# 1時間当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/h) (6f)
def get_W_dash_ba1_d_t(W_ba1_d_t, pipe_diameter):
    """
    :param W_ba1_d_t: 1時間当たりの浴室水栓さし湯時における基準給湯量 (L/h)
    :param pipe_diameter: ヘッダー分岐後の径
    :return: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯量 (L/h)
    """
    # 配管における節湯の効果係数
    f_sp = watersaving.get_f_sp(pipe_diameter)

    return W_ba1_d_t * f_sp

### 10. 基準給湯量

$$\large
\displaystyle
W_{k,d,t} = \left\{ \begin{array}{ll}
    W_{k,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{k,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{k,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{k,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{k,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{k,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{s,d,t} = \left\{ \begin{array}{ll}
    W_{s,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{s,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{s,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{s,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{s,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{s,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{w,d,t} = \left\{ \begin{array}{ll}
    W_{w,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{w,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{w,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{w,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{w,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{w,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ n_{p} $：仮想居住人数(-)  
$ W_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における基準給湯量(L/h)  
$ W_{k,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における台所水栓における基準給湯量(L/h)  
$ W_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における基準給湯量(L/h)  
$ W_{s,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴室シャワー水栓における基準給湯量(L/h)  
$ W_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における基準給湯量(L/h)  
$ W_{w,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における洗面水栓における基準給湯量(L/h)  

In [ ]:
# 1時間当たりの台所水栓における基準給湯量 (L/h) (7a)
def get_W_k_d_t(n_p, schedule_hw):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :return: 1時間当たりの台所水栓における基準給湯量 (L/h)
    """
    if n_p in [1, 2, 3, 4]:
        return get_W_k_p_d_t(n_p, schedule_hw)
    elif 1 <= n_p and n_p <= 2:
        W_k_1_d_t = get_W_k_p_d_t(1, schedule_hw)
        W_k_2_d_t = get_W_k_p_d_t(2, schedule_hw)
        return W_k_1_d_t * (2 - n_p) / (2 - 1) + W_k_2_d_t * (n_p - 1) / (2 - 1)
    elif 2 <= n_p and n_p <= 3:
        W_k_2_d_t = get_W_k_p_d_t(2, schedule_hw)
        W_k_3_d_t = get_W_k_p_d_t(3, schedule_hw)
        return W_k_2_d_t * (3 - n_p) / (3 - 2) + W_k_3_d_t * (n_p - 2) / (3 - 2)
    elif 3 <= n_p and n_p <= 4:
        W_k_3_d_t = get_W_k_p_d_t(3, schedule_hw)
        W_k_4_d_t = get_W_k_p_d_t(4, schedule_hw)
        return W_k_3_d_t * (4 - n_p) / (4 - 3) + W_k_4_d_t * (n_p - 3) / (4 - 3)


# 1時間当たりの浴室シャワー水栓における基準給湯量 (7b)
def get_W_s_d_t(n_p, schedule_hw, has_bath):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param: has_bath: 浴室等の有無
    :return: 1時間当たりの浴室シャワー水栓における基準給湯量 (L/h)
    """
    if n_p in [1, 2, 3, 4]:
        return get_W_s_p_d_t(n_p, schedule_hw, has_bath)
    elif 1 <= n_p and n_p <= 2:
        W_s_1_d_t = get_W_s_p_d_t(1, schedule_hw, has_bath)
        W_s_2_d_t = get_W_s_p_d_t(2, schedule_hw, has_bath)
        return W_s_1_d_t * (2 - n_p) / (2 - 1) + W_s_2_d_t * (n_p - 1) / (2 - 1)
    elif 2 <= n_p and n_p <= 3:
        W_s_2_d_t = get_W_s_p_d_t(2, schedule_hw, has_bath)
        W_s_3_d_t = get_W_s_p_d_t(3, schedule_hw, has_bath)
        return W_s_2_d_t * (3 - n_p) / (3 - 2) + W_s_3_d_t * (n_p - 2) / (3 - 2)
    elif 3 <= n_p and n_p <= 4:
        W_s_3_d_t = get_W_s_p_d_t(3, schedule_hw, has_bath)
        W_s_4_d_t = get_W_s_p_d_t(4, schedule_hw, has_bath)
        return W_s_3_d_t * (4 - n_p) / (4 - 3) + W_s_4_d_t * (n_p - 3) / (4 - 3)


# 1時間当たりの洗面水栓における基準給湯量 (7c)
def get_W_w_d_t(n_p, schedule_hw):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :return: 1時間当たりの洗面水栓における基準給湯量 (L/h)
    """
    if n_p in [1, 2, 3, 4]:
        return get_W_w_p_d_t(n_p, schedule_hw)
    elif 1 <= n_p and n_p <= 2:
        W_w_1_d_t = get_W_w_p_d_t(1, schedule_hw)
        W_w_2_d_t = get_W_w_p_d_t(2, schedule_hw)
        return W_w_1_d_t * (2 - n_p) / (2 - 1) + W_w_2_d_t * (n_p - 1) / (2 - 1)
    elif 2 <= n_p and n_p <= 3:
        W_w_2_d_t = get_W_w_p_d_t(2, schedule_hw)
        W_w_3_d_t = get_W_w_p_d_t(3, schedule_hw)
        return W_w_2_d_t * (3 - n_p) / (3 - 2) + W_w_3_d_t * (n_p - 2) / (3 - 2)
    elif 3 <= n_p and n_p <= 4:
        W_w_3_d_t = get_W_w_p_d_t(3, schedule_hw)
        W_w_4_d_t = get_W_w_p_d_t(4, schedule_hw)
        return W_w_3_d_t * (4 - n_p) / (4 - 3) + W_w_4_d_t * (n_p - 3) / (4 - 3)
    else:
        raise ValueError(n_p)

In [ ]:
# 生活スケジュールパターン
def get_schedule_pattern_list():
    ptn_list = [
        '休日在宅（大）',
        '休日在宅（小）',
        '平日（大）',
        '平日（中）',
        '平日（小）',
        '休日外出'
    ]
    return ptn_list

In [ ]:
# 1時間当たりの居住人数がp人における台所水栓における基準給湯量
def get_W_k_p_d_t(p, schedule_hw):
    """
    :param p: 居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :return: 1時間当たりの居住人数がp人における台所水栓における基準給湯量 (L/h)
    """
    # 読み取るべき表の選択
    table = schedule.get_table_m_for_p(p)

    # 作業用
    W_k_p_d_t = np.zeros(24 * 365)

    # 生活スケジュールパターン
    ptn_list = get_schedule_pattern_list()

    # パターンごとに合算
    for i, ptn in enumerate(ptn_list):
        f = np.repeat(schedule_hw == ptn, 24)
        W_k_p_d_t[f] = np.tile(table[i][:, 0], 365)[f]

    return W_k_p_d_t


# 1時間当たりの居住人数がp人における浴室シャワー水栓における基準給湯量
def get_W_s_p_d_t(p, schedule_hw, has_bath):
    """
    :param p: 居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :return: 1時間当たりの居住人数がp人における洗面シャワー水栓における基準給湯量 (L/h)
    """
    # 読み取るべき表の選択
    table = schedule.get_table_m_for_p(p)

    # 作業用
    W_s_p_d_t = np.zeros(24 * 365)

    # 生活スケジュールパターン
    ptn_list = get_schedule_pattern_list()

    # 表6で読み取るべき列インデックス
    j = 1 if has_bath else 2

    # パターンごとに合算
    for i, ptn in enumerate(ptn_list):
        f = np.repeat(schedule_hw == ptn, 24)
        W_s_p_d_t[f] = np.tile(table[i][:, j], 365)[f]

    return W_s_p_d_t


# 1時間あたりの居住人数がp人における洗面水栓における基準給湯量
def get_W_w_p_d_t(p, schedule_hw):
    """
    :param p: 居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :return: 1日当たりの居住人数がp人における洗面水栓における基準給湯量 (L/d)
    """
    # 読み取るべき表の選択
    table = schedule.get_table_m_for_p(p)

    # 作業用
    W_w_p_d_t = np.zeros(24 * 365)

    # 生活スケジュールパターン
    ptn_list = get_schedule_pattern_list()

    # パターンごとに合算
    for i, ptn in enumerate(ptn_list):
        f = np.repeat(schedule_hw == ptn, 24)
        W_w_p_d_t[f] = np.tile(table[i][:, 3], 365)[f]

    return W_w_p_d_t


$$\large
\displaystyle
W_{b,d,t} = \left\{ \begin{array}{ll}
    W_{b,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{b,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{b,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{b,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{b,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{b,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ n_{p} $：仮想居住人数(-)  
$ W_{b,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽湯はり時における基準給湯量(L/h)  
$ W_{b,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽湯はり時における基準給湯量(L/h)  

In [ ]:
# 浴槽水栓湯はり時における給湯基準量 (L/h)
def get_W_b1_d_t(n_p, schedule_hw, has_bath, bath_function):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :param bath_function: ふろ機能の種類
    :return:浴槽水栓湯はり時における給湯基準量 (L/h)
    """
    if bath_function == '給湯単機能':
        return get_W_b_d_t(n_p, schedule_hw, has_bath)
    elif bath_function == 'ふろ給湯機(追焚なし)' or bath_function == 'ふろ給湯機(追焚あり)':
        return np.zeros(24 * 365)
    else:
        raise ValueError(bath_function)


# 浴槽自動湯はり時における給湯基準量 (L/h)
def get_W_b2_d_t(n_p, schedule_hw, has_bath, bath_function):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :param bath_function: ふろ機能の種類
    :return:浴槽自動湯はり時における給湯基準量 (L/d)
    """
    if bath_function == 'ふろ給湯機(追焚なし)' or bath_function == 'ふろ給湯機(追焚あり)':
        return get_W_b_d_t(n_p, schedule_hw, has_bath)
    elif bath_function == '給湯単機能':
        return np.zeros(24 * 365)
    else:
        raise ValueError(bath_function)


# 1時間当たりの浴槽湯はり時における基準給湯量 (L/h) (8)
def get_W_b_d_t(n_p, schedule_hw, has_bath):
    """
    :param n_p: 仮想居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :return: 1時間あたりの浴槽湯はり時における基準給湯量 (L/h)
    """
    if n_p in [1, 2, 3, 4]:
        return get_W_b_p_d_t(n_p, schedule_hw, has_bath)
    if 1 <= n_p and n_p <= 2:
        W_b_1_d_t = get_W_b_p_d_t(1, schedule_hw, has_bath)
        W_b_2_d_t = get_W_b_p_d_t(2, schedule_hw, has_bath)
        return W_b_1_d_t * (2 - n_p) / (2 - 1) + W_b_2_d_t * (n_p - 1) / (2 - 1)
    elif 2 <= n_p and n_p <= 3:
        W_b_2_d_t = get_W_b_p_d_t(2, schedule_hw, has_bath)
        W_b_3_d_t = get_W_b_p_d_t(3, schedule_hw, has_bath)
        return W_b_2_d_t * (3 - n_p) / (3 - 2) + W_b_3_d_t * (n_p - 2) / (3 - 2)
    elif 3 <= n_p and n_p <= 4:
        W_b_3_d_t = get_W_b_p_d_t(3, schedule_hw, has_bath)
        W_b_4_d_t = get_W_b_p_d_t(4, schedule_hw, has_bath)
        return W_b_3_d_t * (4 - n_p) / (4 - 3) + W_b_4_d_t * (n_p - 3) / (4 - 3)
    else:
        raise ValueError(n_p)


# 1時間あたりの居住人数がp人における浴槽湯はり時における基準給湯量
def get_W_b_p_d_t(p, schedule_hw, has_bath):
    """
    :param p: 居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :return: 1時間あたりの居住人数がp人における浴槽湯はり時における基準給湯量 (L/h)
    """
    # 読み取るべき表の選択
    table = schedule.get_table_m_for_p(p)

    # 作業用
    W_b_p_d_t = np.zeros(24 * 365)

    # 生活スケジュールパターン
    ptn_list = get_schedule_pattern_list()

    # 読み取るべき表の列インデックス
    j = 4 if has_bath else 5

    # パターンごとに合算
    for i, ptn in enumerate(ptn_list):
        f = np.repeat(schedule_hw == ptn, 24)
        W_b_p_d_t[f] = np.tile(table[i][:, j], 365)[f]

    return W_b_p_d_t


# 1時間あたりの居住人数がp人における入浴人数(人/h)
def get_n_b_p_d_t(p, schedule_hw, has_bath):
    """
    :param p: 居住人数 (人)
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :return: 1時間あたりの居住人数がp人における入浴人数(人/h)
    """
    # 読み取るべき表の選択
    table = schedule.get_table_m_for_p(p)

    # 作業用
    n_b_p_d_t = np.zeros(24 * 365)

    # 生活スケジュールパターン
    ptn_list = get_schedule_pattern_list()

    # 読み取るべき表の列インデックス
    j = 6 if has_bath else 7

    # パターンごとに合算
    for i, ptn in enumerate(ptn_list):
        f = np.repeat(schedule_hw == ptn, 24)
        n_b_p_d_t[f] = np.tile(table[i][:, j], 365)[f]

    return n_b_p_d_t

日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量$ W_{ba1,d,t} $は、ふろ機能の種類が給湯単機能又はふろ給湯機（追焚なし）の場合のみに発生する。

$$\large
\displaystyle W_{ba1,d,t} = L_{ba,d,t} \times \frac{1}{(\theta_{sw,ba1} - \theta_{wtr,d})} \times \frac{1}{4.186} \times 10^3
$$

$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ W_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量(L/h)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)  

In [ ]:
# 浴槽水栓さし湯時における基準給湯量 (L/h) (9)
def get_W_ba1_d_t(bath_function, L_ba_d_t, Theta_wtr_d):
    """
    :param bath_function: ふろ機能の種類
    :param L_ba_d_t: 1日当たりの浴槽沸かし直しによる給湯熱負荷 (MJ/h)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 浴槽水栓さし湯時における基準給湯量 (L/h)
    """
    if bath_function == '給湯単機能' or bath_function == 'ふろ給湯機(追焚なし)':
        # 浴槽水栓さし湯時における基準給湯温度
        Theta_sw_ba1 = get_Theta_sw_ba1()
        return L_ba_d_t * (1.0 / (Theta_sw_ba1 - np.repeat(Theta_wtr_d, 24))) * (1.0 / 4.186) * 10 ** 3
    elif bath_function == 'ふろ給湯機(追焚あり)':
        return np.zeros(24 * 365)
    else:
        raise ValueError(bath_function)

### 11. 浴槽沸かし直しによる給湯熱負荷

$$\large
\displaystyle
L_{ba,d,t} = \left\{ \begin{array}{ll}
    L_{ba,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + L_{ba,p,d,t}|_{p=2} \times \displaystyle \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    L_{ba,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + L_{ba,p,d,t}|_{p=3} \times \displaystyle \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    L_{ba,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + L_{ba,p,d,t}|_{p=4} \times \displaystyle \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$ n_{p} $：仮想居住人数(-)  
$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ L_{ba,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/h)  

In [ ]:
# 浴槽沸かし直しによる給湯熱負荷 (MJ/h) (10)
def get_L_ba_d_t(bath_insulation, schedule_hw, has_bath, Theta_ex_Ave_d, n_p):
    """
    :param bath_insulation: 浴槽の断熱の有無
    :param schedule_hw: 給湯スケジュール
    :param has_bath: 浴室等の有無
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :param n_p: 仮想居住人数
    :return: 浴槽沸かし直しによる給湯熱負荷 (MJ/d)
    """
    if 1 <= n_p and n_p <= 2:
        n_b_1_d_t = get_n_b_p_d_t(1, schedule_hw, has_bath)
        n_b_2_d_t = get_n_b_p_d_t(2, schedule_hw, has_bath)
        L_ba_1_d_ = get_L_ba_p_d_t(1, bath_insulation, n_b_1_d_t, Theta_ex_Ave_d)
        L_ba_2_d_t = get_L_ba_p_d_t(2, bath_insulation, n_b_2_d_t, Theta_ex_Ave_d)
        return L_ba_1_d_ * (2 - n_p) / (2 - 1) + L_ba_2_d_t * (n_p - 1) / (2 - 1)
    elif 2 <= n_p and n_p <= 3:
        n_b_2_d_t = get_n_b_p_d_t(2, schedule_hw, has_bath)
        n_b_3_d_t = get_n_b_p_d_t(3, schedule_hw, has_bath)
        L_ba_2_d_t = get_L_ba_p_d_t(2, bath_insulation, n_b_2_d_t, Theta_ex_Ave_d)
        L_ba_3_d_t = get_L_ba_p_d_t(3, bath_insulation, n_b_3_d_t, Theta_ex_Ave_d)
        return L_ba_2_d_t * (3 - n_p) / (3 - 2) + L_ba_3_d_t * (n_p - 2) / (3 - 2)
    elif 3 <= n_p and n_p <= 4:
        n_b_3_d_t = get_n_b_p_d_t(3, schedule_hw, has_bath)
        n_b_4_d_t = get_n_b_p_d_t(4, schedule_hw, has_bath)
        L_ba_3_d_t = get_L_ba_p_d_t(3, bath_insulation, n_b_3_d_t, Theta_ex_Ave_d)
        L_ba_4_d_t = get_L_ba_p_d_t(4, bath_insulation, n_b_4_d_t, Theta_ex_Ave_d)
        return L_ba_3_d_t * (4 - n_p) / (4 - 3) + L_ba_4_d_t * (n_p - 3) / (4 - 3)
    else:
        raise ValueError(n_p)

$$\large
\displaystyle
L_{ba,p,d,t} = \left\{ \begin{array}{ll}
    \big(a_{ba,p,d} \times \theta_{ex,Ave,d} + b_{ba,p,d} \big) \times \frac{n_{b,p,d,t}}{\sum_{t=0}^{23} n_{b,p,d,t}}  & \bigg(\sum_{t=0}^{23} n_{b,p,d,t} > 0 \bigg)\\
    0 & \bigg(\sum_{t=0}^{23} n_{b,p,d,t} = 0 \bigg)\\
    \end{array}
\right.
$$

$L_{ba,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$n_{b,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽入浴人数(人/h)  
$W_{b,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽湯はり時における基準給湯量(L/h)  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃)  
$a_{ba,p,d}, b_{ba,p,d}$：係数(-)  

In [ ]:
# 居住人数がp人における浴槽沸かし直しにおける給湯熱負荷 (11)
def get_L_ba_p_d_t(p, bath_insulation, n_b_p_d_t, Theta_ex_Ave_d):
    """
    :param p: 居住人数 (人)
    :param bath_insulation: 浴槽の断熱の有無
    :param n_b_p_d_t: 居住人数p人における入浴人数(人/h)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :return: 居住人数がp人における浴槽沸かし直しにおける給湯熱負荷 (MJ/d)
    """
    # 係数a_ba, b_ba
    a_ba_p_d, b_ba_p_d = get_coeff_eq11(bath_insulation, p, Theta_ex_Ave_d)

    # 24時間化
    a_ba_p_d = np.repeat(a_ba_p_d, 24)
    b_ba_p_d = np.repeat(b_ba_p_d, 24)
    Theta_ex_Ave_d = np.repeat(Theta_ex_Ave_d, 24)

    # 浴槽沸かし直しによよる給湯熱負荷 (MJ/h) (11)

    # L_ba_p_d_t の作業領域確保
    L_ba_p_d_t = np.zeros(24 * 365)

    # 1日あたりののべ入浴人数
    n_b_p_d = np.repeat(np.sum(n_b_p_d_t.reshape(365, 24), axis=1), 24)

    # W_b_p_d > = 0 の場合
    f1 = (n_b_p_d > 0)
    L_ba_p_d_t[f1] = (a_ba_p_d[f1] * Theta_ex_Ave_d[f1] + b_ba_p_d[f1]) * (n_b_p_d_t[f1] / n_b_p_d[f1])

    # W_b_p_d = 0 の場合
    f2 = (n_b_p_d == 0)
    L_ba_p_d_t[f2] = 0

    return L_ba_p_d_t

<div style="text-align: center;font-weight: bold;">表　係数 $a_{ba}$ 及び $b_{ba}$ の値</div>

<table>
<thead>
<tr>
<th rowspan="2">日平均外気温度の条件</th>
<th rowspan="2">浴槽の種類</th>
<th colspan="2" style="text-align:center;">4 人</th>
<th colspan="2" style="text-align:center;">3 人</th>
<th colspan="2" style="text-align:center;">2 人</th>
<th colspan="2" style="text-align:center;">1 人</th>
</tr>
<tr>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
</tr>
</thead>

<tbody>
<tr><td rowspan="2">7℃未満</td><td>通常浴槽</td><td>-0.12</td><td>6.00</td><td>-0.10</td><td>4.91</td><td>-0.06</td><td>3.02</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.98</td><td>-0.06</td><td>3.22</td><td>-0.04</td><td>2.01</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">7℃以上かつ16℃未満</td><td>通常浴槽</td><td>-0.13</td><td>6.04</td><td>-0.10</td><td>4.93</td><td>-0.06</td><td>3.04</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.08</td><td>4.02</td><td>-0.06</td><td>3.25</td><td>-0.04</td><td>2.03</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">16℃以上かつ25℃未満</td><td>通常浴槽</td><td>-0.14</td><td>6.21</td><td>-0.11</td><td>5.07</td><td>-0.07</td><td>3.13</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.09</td><td>4.19</td><td>-0.07</td><td>3.39</td><td>-0.04</td><td>2.12</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">25℃以上</td><td>通常浴槽</td><td>-0.12</td><td>5.81</td><td>-0.10</td><td>4.77</td><td>-0.06</td><td>2.92</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.80</td><td>-0.06</td><td>3.09</td><td>-0.04</td><td>1.92</td><td>0.00</td><td>0.00</td></tr>
</tbody>

</table>


In [ ]:
# 係数a_ba, b_ba
def get_coeff_eq11(bath_insulation, p, Theta_ex_Ave_d):
    """
    :param bath_insulation: 浴槽の断熱の有無
    :param p: 居住人数 (人)
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :return: 係数a_ba, b_ba
    """
    if bath_insulation == False:
        # 通常浴槽
        y_off = 0
    elif bath_insulation == True:
        # 高断熱浴槽
        y_off = 1
    else:
        raise ValueError(bath_insulation)

    x_off = (4 - p) * 2

    # 7度未満
    tmp_a = ([table_6[y_off][x_off + 0]] * 365) * (Theta_ex_Ave_d < 7.0)
    tmp_b = ([table_6[y_off][x_off + 1]] * 365) * (Theta_ex_Ave_d < 7.0)

    # 7度以上かつ16度未満
    tmp_a = tmp_a + ([table_6[y_off + 2][x_off + 0]] * 365) * (7.0 <= Theta_ex_Ave_d) * (Theta_ex_Ave_d < 16.0)
    tmp_b = tmp_b + ([table_6[y_off + 2][x_off + 1]] * 365) * (7.0 <= Theta_ex_Ave_d) * (Theta_ex_Ave_d < 16.0)

    # 16度以上かつ25度未満
    tmp_a = tmp_a + ([table_6[y_off + 4][x_off + 0]] * 365) * (16.0 <= Theta_ex_Ave_d) * (Theta_ex_Ave_d < 25.0)
    tmp_b = tmp_b + ([table_6[y_off + 4][x_off + 1]] * 365) * (16.0 <= Theta_ex_Ave_d) * (Theta_ex_Ave_d < 25.0)

    # 25度以上
    tmp_a = tmp_a + ([table_6[y_off + 6][x_off + 0]] * 365) * (25.0 <= Theta_ex_Ave_d)
    tmp_b = tmp_b + ([table_6[y_off + 6][x_off + 1]] * 365) * (25.0 <= Theta_ex_Ave_d)

    return tmp_a, tmp_b


# 係数 a_ba, b_ba
table_6 = [
    (-0.12, 6.00, -0.10, 4.91, -0.06, 3.02, 0.00, 0.00),
    (-0.07, 3.98, -0.06, 3.22, -0.04, 2.01, 0.00, 0.00),
    (-0.13, 6.04, -0.10, 4.93, -0.06, 3.04, 0.00, 0.00),
    (-0.08, 4.02, -0.06, 3.25, -0.04, 2.03, 0.00, 0.00),
    (-0.14, 6.21, -0.11, 5.07, -0.07, 3.13, 0.00, 0.00),
    (-0.09, 4.19, -0.07, 3.39, -0.04, 2.12, 0.00, 0.00),
    (-0.12, 5.81, -0.10, 4.77, -0.06, 2.92, 0.00, 0.00),
    (-0.07, 3.80, -0.06, 3.09, -0.04, 1.92, 0.00, 0.00)
]


### 12. 日平均給水温度

$$\large 
\theta_{wtr,d} = max(a_{wtr} \times \theta_{ex,prd,Ave,d} + b_{wtr}, 0.5)
$$

$ \theta_{ex,prd,Ave,d}$：日付$d$を基準とした期間平均外気温度(℃)  
$ \theta_{wtr,d}$：日付$d$における日平均給水温度(℃)  
$ a_{wtr}, b_{wtr} $：日平均給水温度を求める際の回帰係数  

<div style="text-align: center;font-weight: bold;">表　日平均給水温度を求める際の回帰係数の値</div>

|地域の区分|$a_{wtr}$|$b_{wtr}$|
|-|-|-|
|1|0.6639|3.466|
|2|0.6639|3.466|
|3|0.6054|4.515|
|4|0.6054|4.515|
|5|0.8660|1.665|
|6|0.8516|2.473|
|7|0.9223|2.097|
|8|0.6921|7.167|


In [ ]:
# 日平均給水温度 (℃) (12)
def get_Theta_wtr_d(region, Theta_ex_prd_Ave_d):
    """
    :param region: 省エネルギー地域区分
    :param Theta_ex_prd_Ave_d: 期間平均外気温度 (℃)
    :return: 日平均給水温度 (℃)
    """
    # 表 7 日平均給水温度を求める際の回帰係数の値
    table_7 = [
        (0.6639, 3.466),
        (0.6639, 3.466),
        (0.6054, 4.515),
        (0.6054, 4.515),
        (0.8660, 1.665),
        (0.8516, 2.473),
        (0.9223, 2.097),
        (0.6921, 7.167)
    ]
    # 日平均給水温度を求める際の会期係数
    a_wtr, b_wtr = table_7[region - 1]

    # 日平均給水温度 (12)
    Theta_wtr_d = np.clip(a_wtr * Theta_ex_prd_Ave_d + b_wtr, 0.5, None)

    return Theta_wtr_d

$$\large 
\theta_{ex,prd,Ave,d} = \sum^{10}_{k = 1} \theta_{ex,Ave,d-k} / 10
$$

$\theta_{ex,Ave,d-k}$：日付$d$を基準とした$k$日前の日平均外気温度(℃)  
$\theta_{ex,prd,Ave,d}$：日付$d$を基準とした期間平均外気温度(℃)  

In [ ]:
# 期間平均外気温度 (℃) (13)
def get_Theta_ex_prd_Ave_d(Theta_ex_Ave_d):
    """
    :param Theta_ex_Ave_d: 日平均外気温度 (℃)
    :return: 期間平均外気温度 (℃)
    """
    # 10日前までを拡張した配列を作る(最終日は削る=>-1)
    tmp = np.zeros(365 + 10 - 1)
    tmp[0:10] = Theta_ex_Ave_d[-10:]
    tmp[10:] = Theta_ex_Ave_d[0:364]

    # 畳み込み演算
    # 10日分のデータにそれぞれ0.1を掛けて加算する→平均が求まる
    Theta_ex_prd_Ave_d = np.convolve(tmp, [0.1] * 10, mode='valid')

    return Theta_ex_prd_Ave_d

### 13. 日平均外気温度

$$\large 
\theta_{ex,Ave,d} = \sum^{23}_{t = 0} \theta_{ex,d,t} \big/ 24
$$

$ \theta_{ex,Ave,d} $：日付$d$における日平均外気温度(℃)  
$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

In [ ]:
# 日平均外気温度 (℃) (14)
def get_Theta_ex_Ave_d(Theta_ex_d_t):
    """
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 日平均外気温度 (℃)
    """
    # 8760時間の一次配列を365*24の二次配列へ再配置させる
    tmp = Theta_ex_d_t.reshape(365, 24)

    # 二次元目を加算することで二次元目を消滅させる
    tmp = np.sum(tmp, axis=1)

    # 24で割ることで平均化する
    Theta_ex_Ave_d = tmp / 24

    return Theta_ex_Ave_d

### 14. 夜間平均外気温度

$$\large 
\theta_{ex,Nave,d} = \bigg\{ \theta_{ex,d-1,23} + \sum^6_{t=0}\theta_{ex,d,t} \bigg\} \big/8
$$

$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$ \theta_{ex,d-1,23} $：日付$d-1$における23 時における外気温度(℃)  
$ \theta_{ex,Nave,d}$：日付$d$における夜間平均外気温度(℃)  

In [ ]:
# 夜間平均外気温度 (℃) (15)
def get_Theta_ex_Nave_d(Theta_ex_d_t):
    """
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 夜間平均外気温度 (℃)
    """
    # 1時間後ろに配列をずらす(そして、12月31日23時を1月1日0時に移動させる)
    tmp = np.roll(Theta_ex_d_t, 1)

    # ** 1時間ずらしたので、前日23時から当日7時までの代わりに、当日0時から8時までの平均を計算すればよい **

    # 8760時間の一次配列を365*24の二次配列へ再配置させる
    tmp = tmp.reshape(365, 24)

    # 8時～23時を0にする
    tmp[:, 8:] = 0

    # 配列の2次元目を合算して2次元目を消す
    tmp = np.sum(tmp, axis=1)

    # 8で割ることで平均化する
    Theta_ex_Nave_d = tmp / 8

    return Theta_ex_Nave_d

### 15. 温水暖房の熱負荷

省略

In [ ]:
# 1日当たりの温水温度の熱負荷 (MJ/d) (16)
def get_L_HWH_d(L_HWH_d_t):
    """
    :param L_HWH_d_t: 1時間当たりの温水暖房の熱負荷 (MJ/d)
    :return: 1日当たりの温水暖房の熱負荷 (MJ/d)
    """
    # 8760時間の一次配列を365*24の二次配列へ再配置させる
    tmp = L_HWH_d_t.reshape(365, 24)

    # 二次元目を加算することで二次元目を消滅させる
    L_HWH_d = np.sum(tmp, axis=1)

    return L_HWH_d

## 付録 A 複数の給湯機を設置する場合の評価において代表させる給湯機の決定方法 

省略

## 付録 B 給湯機を設置しない場合に評価において想定する機器の種類と仕様 

省略

In [ ]:
def get_virtual_hotwater(region, HW):
    if HW is None:
        return None

    # その他または設置しない場合
    if HW['hw_type'] in ['その他', '設置しない', '不明']:
        default = get_default_hw_type(region)
        virt = copy.deepcopy(HW)
        virt['hw_type'] = default[0]
        virt['e_rtd'] = default[1]
        return virt
    else:
        return HW


# 給湯機を設置しない場合に評価において想定する機器の種類と仕様
def get_default_hw_type(region):
    """
    :param region: 省エネルギー地域区分
    :return: 給湯機を設置しない場合に評価において想定する機器の種類と効率
    """
    # 表 B.1 給湯機を設置しない場合の評価において想定する機器
    table_b_1 = [
        ('石油従来型給湯機', 0.813),
        ('石油従来型給湯機', 0.813),
        ('石油従来型給湯機', 0.813),
        ('石油従来型給湯機', 0.813),
        ('ガス従来型給湯機', 0.782),
        ('ガス従来型給湯機', 0.782),
        ('ガス従来型給湯機', 0.782),
        ('ガス従来型給湯機', 0.782),
    ]
    return table_b_1[region - 1]